<a href="https://colab.research.google.com/github/pablomiralles22/class-CV-computer-vision/blob/main/Convolution_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
# --- Data ---
transform = transforms.ToTensor()
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, transform=transform),
    batch_size=64, shuffle=False
)

In [ ]:
# --- Model ---
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2),   # [B, 1, 28, 28] -> [B, 16, 13, 13]
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2),  # [B, 16, 13, 13] -> [B, 32, 6, 6]
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2),  # [B, 32, 6, 6] -> [B, 64, 2, 2]
            nn.ReLU(),
        )
        self.classifier = nn.Linear(64 * 2 * 2, 10)  # Flatten: [B, 256] -> [B, 10]

    def forward(self, x):
        x = self.net(x)
        x = x.flatten(start_dim=1)
        return self.classifier(x)

In [ ]:
# --- Training ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


for epoch in range(5):  # Single epoch for minimal example
    print(f"Epoch {epoch}")

    # Train
    model.train()
    train_losses = []
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    print(f"Train loss: {sum(train_losses) / len(train_losses)}")

    # Validation
    val_losses = []
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_losses.append(loss.item())
    print(f"Val loss: {sum(val_losses) / len(val_losses)}")


Epoch 0
Train loss: 0.36047024827108964
Val loss: 0.14033805436801736
Epoch 1
Train loss: 0.11889521469538814
Val loss: 0.08872888061984674
Epoch 2
Train loss: 0.08452595395273341
Val loss: 0.06738804081850834
Epoch 3
Train loss: 0.06646266297550439
Val loss: 0.06254863348225233
Epoch 4
Train loss: 0.05486489250709408
Val loss: 0.05864701733620805
